In [3]:
!pip install PyPDF2==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd
from sqlalchemy import create_engine
from PyPDF2 import PdfReader
from io import StringIO

In [9]:
db_username = 'lahad'
db_password = 'Lahad1998!'
db_host = 'postgresql-lahad.alwaysdata.net' 
db_port = '5432'
db_name = 'lahad_airport-data'

In [10]:
db_connection_url = f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'

In [11]:
engine = create_engine(db_connection_url)

In [6]:
airport_data = pd.read_excel('airports.xlsx')
flights_data = pd.read_excel('flights.xlsx')
planes_data = pd.read_html('planes.html')[0]
airlines_data = pd.read_json('airlines.json')
pdf_weather_data = PdfReader('weather.pdf')

In [28]:
 text = ''
for page in pdf_weather_data.pages:
    text += page.extract_text()

lines = text.split('\n')  # Séparer le texte par ligne
cleaned_lines = []

# Supprimer les lignes vides ou mal formatées
for line in lines:
    if len(line.split(',')) == 15:  # Vérifier si le nombre de colonnes est correct (15 colonnes)
        cleaned_lines.append(line)

# Rejoindre les lignes nettoyées
cleaned_text = "\n".join(cleaned_lines)
df_weather_data = pd.read_csv(StringIO(cleaned_text))

df_weather_data.head()

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2021,1,1,0,37.04,21.92,53.97,230,10.357019999999999,11.918651475599997,0.0,1013.9,10.0,2021-01-01T00:00:00Z
1,EWR,2021,1,1,1,37.04,21.92,53.97,230,13.809359999999998,15.891535300799996,0.0,1013,10.0,2021-01-01T01:00:00Z
2,EWR,2021,1,1,2,37.94,21.92,52.09,230,12.658579999999999,14.567240692399997,0.0,1012.6,10.0,2021-01-01T02:00:00Z
3,EWR,2021,1,1,3,37.94,23.00,54.51,230,13.809359999999998,15.891535300799996,0.0,1012.7,10.0,2021-01-01T03:00:00Z
4,EWR,2021,1,1,4,37.94,24.08,57.04,240,14.960139999999999,17.215829909199996,0.0,1012.8,10.0,2021-01-01T04:00:00Z


In [29]:
dataframes = {
    'airport': airport_data,
    'flights': flights_data,
    'planes': planes_data,
    'airlines': airlines_data,
    'weather': df_weather_data
}

In [30]:
for table_name, dataframe in dataframes.items():
    dataframe.to_sql(table_name, engine, index=False, if_exists='replace')
    print(f'Table {table_name} has been written to the database.')

Table weather has been written to the database.


In [10]:
# Exécuter une requête avec Pandas
query = "SELECT * FROM airport"
df_airport = pd.read_sql(query, engine)

In [11]:
df_airport

,Unnamed: 0,faa,name,lat,lon,alt,tz,dst,tzone
0,0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
...,...,...,...,...,...,...,...,...,...
1453,1453,ZUN,Black Rock,35.083228,-108.791778,6454,-7,A,America/Denver
1454,1454,ZVE,New Haven Rail Station,41.298669,-72.925992,7,-5,A,America/New_York
1455,1455,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0,-5,A,America/New_York
1456,1456,ZWU,Washington Union Station,38.897460,-77.006430,76,-5,A,America/New_York


In [12]:
query = "SELECT * FROM flights"
df_flights = pd.read_sql(query, engine)
df_flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2021,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2021-01-01T05:00:00Z
1,2021,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2021-01-01T05:00:00Z
2,2021,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2021-01-01T05:00:00Z
3,2021,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2021-01-01T05:00:00Z
4,2021,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2021-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252699,2021,7,3,1059,1100,-1,1214,1210,4,MQ,3230,N522MQ,JFK,DCA,51,213,11,0,2021-07-03T11:00:00Z
252700,2021,7,3,1059,1100,-1,1207,1213,-6,US,2171,N754UW,LGA,DCA,43,214,11,0,2021-07-03T11:00:00Z
252701,2021,7,3,1101,1110,-9,1305,1240,25,9E,3910,N813AY,JFK,RIC,61,288,11,10,2021-07-03T11:00:00Z
252702,2021,7,3,1105,1055,10,1435,1358,37,B6,971,N598JB,LGA,FLL,176,1076,10,55,2021-07-03T10:00:00Z


In [8]:
query = "SELECT * FROM planes"
df_planes = pd.read_sql(query, engine)
df_planes

,Unnamed: 0,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan
...,...,...,...,...,...,...,...,...,...,...
3317,3317,N997AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan
3318,3318,N997DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2,142,NaN,Turbo-fan
3319,3319,N998AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan
3320,3320,N998DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS CORPORATION,MD-88,2,142,NaN,Turbo-jet


Nomenclature de la clé faa de la table airport

In [41]:
df_2 = (lambda: _deepnote_execute_sql('SELECT faa \nFROM airport \nWHERE faa !~ \'^[A-Z0-9]{3}$\';\n', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT faa \nFROM airport \nWHERE faa !~ \'^[A-Z0-9]{3}$\';\n', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_2

,faa


- Vérification des avions (tailnum) qui n'existent pas dans planes :

In [67]:
df_3 = (lambda: _deepnote_execute_sql('SELECT DISTINCT tailnum \nFROM flights \nWHERE tailnum NOT IN (SELECT tailnum FROM planes);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT DISTINCT tailnum \nFROM flights \nWHERE tailnum NOT IN (SELECT tailnum FROM planes);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_3

,tailnum


- Vérification des compagnies aériennes (carrier) qui n'existent pas dans airlines :

In [48]:
df_4 = (lambda: _deepnote_execute_sql('SELECT DISTINCT carrier \nFROM flights \nWHERE carrier NOT IN (SELECT carrier FROM airlines);\n', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT DISTINCT carrier \nFROM flights \nWHERE carrier NOT IN (SELECT carrier FROM airlines);\n', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_4

,carrier


### Les clés primaires

Requête pour définir faa comme clé primaire de la table airport

In [36]:
alter_query_airport = """
    ALTER TABLE airport
    ADD PRIMARY KEY (faa);
    """
engine.execute(alter_query_airport)

•	Requête  pour définir 'tailnum' est la PK de la table planes 

In [9]:
alter_query_planes = """
    ALTER TABLE planes
    ADD PRIMARY KEY (tailnum);
    """

engine.execute(alter_query_planes)


- Requête  pour définir 'carrier' comme clé primaire dans la table airlines

In [10]:
alter_query_airlines = """
    ALTER TABLE airlines
    ADD PRIMARY KEY (carrier);
    """

engine.execute(alter_query_airlines)

- Requête pour définir une clé primaire composite dans la table weather

In [32]:
alter_query_weather = """
    ALTER TABLE weather
    ADD PRIMARY KEY (year, month, day, hour, origin);
    """
engine.execute(alter_query_weather)

- Vérifier que ni le numéro de vol ni la combinaison de la date avec le numéro du vol et/ou le tailnum donne une PK pour la table flights

In [16]:
# 1. Vérifier l'unicité du numéro de vol
unique_flight_query = """
    SELECT flight, COUNT(*)
    FROM flights
    GROUP BY flight
    HAVING COUNT(*) > 1 ;
    """
duplicate_flights = engine.execute(unique_flight_query).fetchall()
if duplicate_flights:
        print("Il y a des doublons dans le numéro de vol (flight).")
else:
    print("Tous les numéros de vol sont uniques.")

Il y a des doublons dans le numéro de vol (flight).


- Vérifier l'unicité de la combinaison (year, month, day, flight, tailnum)

In [24]:
# 2. Vérifier l'unicité de la combinaison (year, month, day, flight, tailnum)
unique_combination_query = """
    SELECT year, month, day, flight, tailnum, COUNT(*)
    FROM flights
    GROUP BY year, month, day, flight, tailnum
    HAVING COUNT(*) > 1;
    """
    
duplicate_combinations = engine.execute(unique_combination_query).fetchall()
if duplicate_combinations:
    print("Il y a des doublons dans la combinaison (year, month, day, flight, tailnum).")
else:
    print("Toutes les combinaisons sont uniques.")

Il y a des doublons dans la combinaison (year, month, day, flight, tailnum).


- Ajouter une clé primaire artificielle auto-incrémentée si nécessaire

In [31]:
 # 3. Ajouter une clé primaire artificielle auto-incrémentée si nécessaire
if duplicate_flights or duplicate_combinations:
    alter_query = """
    ALTER TABLE flights
    ADD COLUMN flight_id SERIAL PRIMARY KEY;
    """

    engine.execute(alter_query)
    print("Une clé primaire artificielle 'flight_id' a été ajoutée à la table 'flights'.")


Une clé primaire artificielle 'flight_id' a été ajoutée à la table 'flights'.


4.

In [ ]:
df_5 = (lambda: _deepnote_execute_sql('INSERT INTO planes(tailnum) SELECT DISTINCT tailnum FROM flights WHERE tailnum NOT IN (SELECT tailnum FROM planes);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('INSERT INTO planes(tailnum) SELECT DISTINCT tailnum FROM flights WHERE tailnum NOT IN (SELECT tailnum FROM planes);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_5

In [65]:
flights_airplanes = """
ALTER TABLE flights
ADD CONSTRAINT fk_flights_tailnum
FOREIGN KEY (tailnum) REFERENCES planes(tailnum);
"""

engine.execute(flights_airplanes)

ProgrammingError: (psycopg2.errors.DuplicateObject) constraint "fk_flights_tailnum" for relation "flights" already exists

[SQL: 
ALTER TABLE flights
ADD CONSTRAINT fk_flights_tailnum
FOREIGN KEY (tailnum) REFERENCES planes(tailnum);
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [66]:
flights_airlines = """
ALTER TABLE flights
ADD CONSTRAINT fk_flights_carrier
FOREIGN KEY (carrier) REFERENCES airlines(carrier);
"""  # Fixed table reference from planes to airlines

engine.execute(flights_airlines)

ProgrammingError: (psycopg2.errors.DuplicateObject) constraint "fk_flights_carrier" for relation "flights" already exists

[SQL: 
ALTER TABLE flights
ADD CONSTRAINT fk_flights_carrier
FOREIGN KEY (carrier) REFERENCES airlines(carrier);
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [68]:
flights_airports_origin = """
ALTER TABLE flights
ADD CONSTRAINT fk_flights_origin
FOREIGN KEY (origin) REFERENCES airport(faa);
"""

engine.execute(flights_airports_origin)

In [ ]:
df_6 = (lambda: _deepnote_execute_sql('INSERT INTO airport(faa) SELECT DISTINCT dest FROM flights WHERE dest NOT IN (SELECT faa FROM airport);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('INSERT INTO airport(faa) SELECT DISTINCT dest FROM flights WHERE dest NOT IN (SELECT faa FROM airport);', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_6

In [74]:
flights_airports_dest = """
ALTER TABLE flights
ADD CONSTRAINT fk_flights_dest
FOREIGN KEY (dest) REFERENCES airport(faa);
"""

engine.execute(flights_airports_dest)

In [75]:
weather_airports = """
ALTER TABLE weather
ADD CONSTRAINT fk_weather_origin
FOREIGN KEY (origin) REFERENCES airport(faa);
"""

engine.execute(weather_airports)

In [34]:
df_1 = (lambda: _deepnote_execute_sql('SELECT year, month, day, hour, flight, carrier, COUNT(*)\n    FROM flights\n    GROUP BY year, month, day, hour, flight, carrier\n    HAVING COUNT(*) > 1;', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT year, month, day, hour, flight, carrier, COUNT(*)\n    FROM flights\n    GROUP BY year, month, day, hour, flight, carrier\n    HAVING COUNT(*) > 1;', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_1

,year,month,day,hour,flight,carrier,count


In [76]:
df_7 = (lambda: _deepnote_execute_sql('SELECT * FROM planes;', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT * FROM planes;', 'SQL_CD1AB1D5_AAEA_4C24_8A61_F14AD02B7B7C', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_7

,Unnamed: 0,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0.0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2.0,55.0,NaN,Turbo-fan
1,1.0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan
2,2.0,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan
3,3.0,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2.0,182.0,NaN,Turbo-fan
4,4.0,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2.0,55.0,NaN,Turbo-fan
...,...,...,...,...,...,...,...,...,...,...
4034,NaN,N4WKAA,NaN,None,None,None,NaN,NaN,NaN,None
4035,NaN,N630MQ,NaN,None,None,None,NaN,NaN,NaN,None
4036,NaN,N354AT,NaN,None,None,None,NaN,NaN,NaN,None
4037,NaN,N345AA,NaN,None,None,None,NaN,NaN,NaN,None


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36af6df1-744c-45ad-80dd-7efe5fea58f5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>